[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.  

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

In [ ]:
# Import pandas, numpy
import pandas as pd
import numpy as np

#### Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

Objective of this dataset is base on **review** we predict **sentiment** so X is **review** column and y is **sentiment** column

In [ ]:
# Read dataset
dataset = pd.read_csv("./labeledTrainData.tsv", sep='\t', encoding="latin-1")

dataset.sample(10)

,id,sentiment,review
4185,7223_10,1,"The comparison to Sleuth, the earlier stage-pl..."
10873,9428_10,1,One of my favorite movies to date starts as an...
18437,4765_7,1,A priest who has abandoned his ministry meets ...
5471,6522_10,1,During the whole Pirates of The Caribbean Tril...
16585,6245_10,1,"\Hundstage\"" is seidl's first fiction film (be..."
13918,8118_3,0,"Well, EYEboy, I must say that it pleases me to..."
631,2025_1,0,My wife and I are semi amused by Howie Mandel'...
12895,2124_1,0,This film concerns the story of Eddy as mentio...
20416,9265_9,1,Different film directors from different countr...
1937,2808_10,1,"This film is not morbid, nor is it depressing...."


## 2. Preprocessing

In [ ]:
# stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tantran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    
    # Save emoticons for later appending
    # If we do not use the non-capturing pattern '?:' for the groups, the returned result is a list of tuples, each tuple represents for each group. 
    # If we use non-capturing grouping '(?:)',that is, we group but not capture them, we can return a string
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Remove any non-word character(including white spaces) and append the emoticons,
    # Removing the nose character for standarization. Convert to lower case
    # If we don't use Quantifier '+', we only search for one character (non-alphanumeric character)
    # What this Regex do is Selecting a sequence of non-alphanumeric characters
    # We use join() because the 'emoticons' is actually a list
    # We replace '-' with '' because ':-)' is the same as ':)'
    text = re.sub('([\W]+)(d|p)*', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    
    return text

preprocessor(':) Today is fine :-D :P')


' today is fine  :) :D :P'

In [ ]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root 
from nltk.stem import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    stem_tokens=[]
    tokens = text.split(' ')
    for token in tokens:
        stem_tokens.append(porter.stem(token))
    return stem_tokens

In [ ]:
# split the dataset in train and test
from sklearn.model_selection import train_test_split
X = dataset['review']
y = dataset['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## 3. Create Model and Train 

In [ ]:
# Using Pipeline to concat tfidf step and LogisticRegression step
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# If we don't pass these params, defaultly they are None
# If we don't pass stop_words, and preprocessor, our data is not good enough
# If we don't pass tokenizer, TfidfVectorizer will use the default one, but here we have tokenizer combined with stemming, better than default
tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

## 4. Evaluate Model

In [ ]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

prediction = clf.predict(X_test)
matrix = confusion_matrix(y_test, prediction)
report = classification_report(y_test, prediction)
print(matrix)
print(report)

[[3294  502]
 [ 357 3347]]
             precision    recall  f1-score   support

          0       0.90      0.87      0.88      3796
          1       0.87      0.90      0.89      3704

avg / total       0.89      0.89      0.89      7500



## 5. Export Model 

In [ ]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open(os.path.join('data', '../movieReviewSentiments.pkl'), 'wb'), protocol=4)

FileNotFoundError: [Errno 2] No such file or directory: 'data/../movieReviewSentiments.pkl'